In [1]:
import numpy as np
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import os
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")

In [2]:
engine = create_engine('postgresql+psycopg2://{user}:{pw}@{service}:{port}/{db}'.format(
    user = 'postgres',
    pw = POSTGRES_PASSWORD,
    service = 'postgres',
    port = '5432',
    db = 'nba'
))

In [3]:
myquery = '''
SELECT 
FROM games
WHERE season = 2022
LIMIT 20

'''

pd.read_sql_query(myquery, con = engine)

""


### ADDITIONAL SQL COMMANDS
INNER JOIN \
    ON \
ORDER BY ot\
    DESC \
LIMIT 20

From the player_game table, make a table with all players who scored more than 30 points in the game. Keep only the columns with the game and player id, points, and three point shots made and attempted. Sort by points in descending order, and display just the top 15. 

In [4]:
myquery = '''
SELECT game_id as game, player_id as player, fg3 as field_goal, fg3a as field_goal_attempted, pts as points
FROM player_game
WHERE pts >= 30
ORDER BY points
    DESC
LIMIT 15
'''

pd.read_sql_query(myquery, con = engine)

,game,player,field_goal,field_goal_attempted,points
0,202101030GSW,curryst01,8,16,62
1,202001200POR,lillada01,11,20,61
2,202008110DAL,lillada01,9,17,61
3,202008110DAL,lillada01,9,17,61
4,202001200POR,lillada01,11,20,61
5,201911300HOU,hardeja01,8,14,60
6,202101060PHI,bealbr01,7,10,60
7,202104300BOS,tatumja01,5,7,60
8,201911080POR,lillada01,7,16,60
9,201911080POR,lillada01,7,16,60


In [5]:
myquery = '''
SELECT g.game_date, p.player, pg.pts, pg.fg3, pg.fg3a
FROM player_game pg
INNER JOIN players p 
    ON pg.player_id = p.player_id
INNER JOIN games g
    ON pg.game_id = g.game_id
WHERE pts > 30
ORDER BY pg.pts DESC
LIMIT 15
'''

pd.read_sql_query(myquery, con = engine)

,game_date,player,pts,fg3,fg3a
0,2021-01-03,Stephen Curry,62,8,16
1,2020-08-11,Damian Lillard,61,9,17
2,2020-01-20,Damian Lillard,61,11,20
3,2020-01-20,Damian Lillard,61,11,20
4,2020-08-11,Damian Lillard,61,9,17
5,2021-04-30,Jayson Tatum,60,5,7
6,2019-11-08,Damian Lillard,60,7,16
7,2021-01-06,Bradley Beal,60,7,10
8,2021-04-30,Jayson Tatum,60,5,7
9,2021-01-06,Bradley Beal,60,7,10


In the team games table, who won each game? 
That will require you to join the team game to itself! :D \
Keep the game id, the team and opp. abr., the team score, and the other team's score renamed to "opponent score"

In [14]:
myquery = '''
SELECT *
FROM team_game t

'''

pd.read_sql_query(myquery, con = engine)

,game_id,Team_Abbrev,H_A,Team_Score,Team_pace,Team_efg_pct,Team_tov_pct,Team_orb_pct,Team_ft_rate,Team_off_rtg,Opponent_Abbrev
0,202202170BRK,WAS,A,117,94.5,0.627,13.5,22.9,0.157,123.8,BRK
1,202202170BRK,BRK,H,103,94.5,0.483,13.1,33.3,0.191,109.0,WAS
2,202202170CHO,MIA,A,111,88.8,0.471,11.1,26.8,0.147,103.4,CHO
3,202202170CHO,CHO,H,107,88.8,0.453,13.6,28.1,0.221,99.7,MIA
4,202202170LAC,HOU,A,111,103.7,0.533,15.3,24.0,0.154,107.1,LAC
...,...,...,...,...,...,...,...,...,...,...,...
6389,202002270GSW,GSW,H,86,104.8,0.481,23.6,12.2,0.113,82.1,LAL
6390,202002290PHO,GSW,A,115,98.6,0.523,9.0,28.9,0.276,116.6,PHO
6391,202003010GSW,GSW,H,110,100.2,0.522,17.4,38.3,0.191,109.8,WAS
6392,202003030DEN,GSW,A,116,94.4,0.622,10.7,12.8,0.171,122.9,DEN


In [16]:
myquery = '''
SELECT t.game_id,
    t.Team_Abbrev,
    t.Opponent_Abbrev,
    t.team_score,
    s.team_score AS opponent_score
FROM team_game t
INNER JOIN team_game s
    ON t.game_id = s.game_id
        AND t.Team_Abbrev = s.Opponent_Abbrev
'''
pd.read_sql_query(myquery, con=engine)

ProgrammingError: (psycopg2.errors.UndefinedColumn) column t.team_abbrev does not exist
LINE 10:         AND t.Team_Abbrev = s.Opponent_Abbrev
                     ^
HINT:  Perhaps you meant to reference the column "t.Team_Abbrev".

[SQL: 
SELECT t.game_id,
    t.Team_Abbrev,
    t.Opponent_Abbrev,
    t.team_score,
    s.team_score AS opponent_score
FROM team_game t
INNER JOIN team_game s
    ON t.game_id = s.game_id
        AND t.Team_Abbrev = s.Opponent_Abbrev
]
(Background on this error at: https://sqlalche.me/e/20/f405)